# Alignment by Dynamic Programming

Uses dynamic programming method of solving problem by breaking it down into simpler sub-problems in a recursive manner.

## Imports

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_import = pd.read_pickle("/Users/traceyetheridge/Documents/Project/Git_Shared/data/df_clean_newwer_new.pkl")
df_import.shape

(333459, 10)

In [3]:
df_import.head(3)

,index,identifier,speaker_id,file,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer
0,1,sw2061A-ms98-a-0123,1167,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,yeah because see what happens is they have a g...,yeah because see what happens is they have a g...,0.000000
1,6,sw4522A-ms98-a-0011,1601,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,they treat them they treat them just utterly h...,they treat them they treat them just utterly h...,0.105263
2,8,sw2335B-ms98-a-0086,1175,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,stuffed them in the freezer and then shot them...,stuff the minute freezing they shot them all,0.600000


In [4]:
identifiers = df_import['identifier'].unique()
len(identifiers)

21391

## Functions

### word_distance
Checks if 2 words match. Currently based on exact match but can be adjusted to Levehnstein distance if desired.
- Returns 0 for match or 1 for no match. 

In [5]:
def word_distance(word0, word1):
  """Return 0 for match 1 otherwise"""
  return int(word0 != word1)

### cost_matrix_sent
Creates cost matrix for 2 sentences.
- Step1: Initialise 0 matrix for length of sentences
- Step2: Compare each word from sentences and populate matrix with word distance value

In [6]:
def cost_matrix_sent(sent0, sent1):
    cost_matrix = np.zeros([len(sent0), len(sent1)])
    for i, word0 in enumerate(sent0):
        for j, word1 in enumerate(sent1):
            cost_matrix[i, j] = word_distance(word0, word1)
    return cost_matrix

### min_cost_matrix

Now we accumulate the minimum cost by assuming we can only move right (word in column sentence doesn't match row sentence), down (word in row sentence doesn't match col sentence) or a match.

The `min_cost_matrix` shows the minimum amount of cost to get from the start position of the matrix to the word alignment. 

Now we have the minimum cost we work our way backwards to get back to the start of both sentences. The resulting `path` traces out the slignment.

In [7]:
def make_min_cost_matrix(cost_matrix, skip_cost=.45):
    min_cost_matrix = np.zeros_like(cost_matrix)
    move_matrix_i = np.zeros(cost_matrix.shape, np.int)
    move_matrix_j = np.zeros(cost_matrix.shape, np.int)
    move_matrix_i[1:, 0] = -1
    move_matrix_j[0, 1:] = -1

    min_cost_matrix[0, :] = np.cumsum(cost_matrix[0, :])*skip_cost
    min_cost_matrix[:, 0] = np.cumsum(cost_matrix[:, 0])*skip_cost

    if cost_matrix.shape[1] == 1:
        path = 'one_word'

    else:
        for i in range(1, cost_matrix.shape[0]):
            for j in range(1, cost_matrix.shape[1]):
              # min of up or left
                if min_cost_matrix[i-1, j] <= min_cost_matrix[i, j-1]:
                    move_matrix_i[i, j] = -1
                    min_cost_matrix[i, j] = min_cost_matrix[i-1, j] + skip_cost
                else:
                    move_matrix_j[i, j] = -1
                    min_cost_matrix[i, j] = min_cost_matrix[i, j-1] + skip_cost

                # min of previous(up or left) and diag+penalty
                diag_cost = min_cost_matrix[i-1, j-1]+cost_matrix[i, j]
                if diag_cost < min_cost_matrix[i, j]:
                    move_matrix_i[i, j] = -1
                    move_matrix_j[i, j] = -1
                    min_cost_matrix[i, j] = min_cost_matrix[i-1, j-1]+cost_matrix[i, j]

        path = [[i, j]]
        while (i > 0) or (j > 0):
            i += move_matrix_i[i, j]
            j += move_matrix_j[i, j]
            path = [[i,j]] + path

    return np.array(path)

### merge

Create combination sentence:
- checks words at each set of path coordinates for match
- if match found add one word to combination sentence
- if match is not found add both words to combination sentence



In [8]:
# Create combination sentence

def merge(s0_words, s1_words, path):
    
    if path == 'one_word':
        words = s0_words
        if s1_words not in s0_words:
            words.append(s1_words[0])
    
    else:
        old_p = np.zeros_like(path[0])
        words = [s1_words[path[0][1]]]
        if s0_words[path[0][0]] != s1_words[path[0][1]]:
            words.append(s0_words[path[0][0]])

        for p in path[1:]:
            advance = p - old_p
            if advance[0]:
                words.append(s0_words[p[0]])
                if advance[1] and (s1_words[p[1]] not in s0_words):
                    words.append(s1_words[p[1]])
            if advance[1] and not advance[0]:
                words.append(s1_words[p[1]])
            old_p = p
            
        if path[-1][0] == path[-1][1]:
            if s0_words[path[-1][0]] != s1_words[path[-1][1]]:
                words.append(s1_words[path[-1][1]])
            
    return words

# Make loop to create new combination sentence as each new hypothesis text is added

def all_text_merge(s0, s1, path, df):
    words = merge(s0, s1, path)
    for row in range(1,len(df)):
        sent = df[row].split(' ')
        cost_matrix = cost_matrix_sent(words,sent)
        path = make_min_cost_matrix(cost_matrix)
        words = merge(words, sent, path)
    return words

### create_aligned_dataframe
Create dataframe with aligned words
- Create empty dataframe of length = length of combination words previously created
- Loop through each sentence and place each word from sentence in column aligned to combination words

In [9]:
def create_aligned_dataframe(df, words):
    rows = []
    for i in range(len(df)):
        row = ['']*len(words)
        sent = df[i].split(' ')
        sent_idx = 0
        for w, word in enumerate(words):
            if sent[sent_idx] == word:
                row[w] = word
                sent_idx += 1
            if sent_idx == len(sent):
                break
        rows.append(row)

    df_aligned = pd.DataFrame(rows)
    return df_aligned

### align_hyp
Put it all together

In [10]:
def align_hyp(df):
    s0 = df[0].split(' ')
    s1 = df[1].split(' ')

    cost_matrix = cost_matrix_sent(s0,s1)

    path = make_min_cost_matrix(cost_matrix)

    words = all_text_merge(s0, s1, path, df)
    
    df_aligned = create_aligned_dataframe(df, words)
    
    return df_aligned

# Example

### Selection a set of hypothesis texts

In [11]:
df_test = df_import[df_import['identifier'] == df_import['identifier'].iloc[25]]
df_test = df_test.drop(['file', "speaker_id"], axis=1)
df_temp = df_test['hypothesis.text'].reset_index(drop=True)
df_temp

0     based on prices and stuff and then of course w...
1     based on prices and stuff and then of course w...
2     based on prices and stuff and of course for ho...
3     based on prices and stuff and of course for ho...
4     based on prices and stuff and of course we're ...
5     based on prices and stuff and then of course w...
6     play some prices of stuff in its course with h...
7     play some prices of stuff in its course with h...
8     based on prices and stuff and of course were f...
9     based on prices and stuff and of course were f...
10    right course for home repair anything like tha...
11    right course for home repair anything like tha...
12    based on price of the staff in the course with...
13                                              i think
14                                                 what
Name: hypothesis.text, dtype: object

### Run through alignment algorithn

In [12]:
pd.set_option('display.max_columns', 500)
df_aligned = align_hyp(df_temp)
df_aligned

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,,based,,,,,,on,,prices,,,,and,stuff,,,,and,then,of,course,,,,we're,home,repair,,,or,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,or,fixing,,,,something,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
1,,based,,,,,,on,,prices,,,,and,stuff,,,,and,then,of,course,,,,we're,home,repair,,,or,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,or,fixing,,,,something,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
2,,based,,,,,,on,,prices,,,,and,stuff,,,,and,,of,course,,,for,,home,repair,,,or,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,if,i,do,plan,on,building,a,door,or,fixing,,,,something,i,would,go,to,,library,,,and,,start,preparing,for,,before,i,did,,that,
3,,based,,,,,,on,,prices,,,,and,stuff,,,,and,,of,course,,,for,,home,repair,,,or,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,if,i,do,plan,on,building,a,door,or,fixing,,,,something,i,would,go,to,,library,,,and,,start,preparing,for,,before,i,did,,that,
4,,based,,,,,,on,,prices,,,,and,stuff,,,,and,,of,course,,,,we're,home,repair,,,or,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,or,fixing,,,,something,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
5,,based,,,,,,on,,prices,,,,and,stuff,,,,and,then,of,course,,,,we're,home,repair,,,,anything,like,that,i,wouldn't,do,them,any,prejudice,,,,anyway,so,,i,do,plan,on,building,a,door,,fixing,,,,something,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
6,,,,,play,,some,,,prices,of,,,,stuff,in,,its,,,,course,,with,,,home,repair,,,,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,,fixing,,some,,,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
7,,,,,play,,some,,,prices,of,,,,stuff,in,,its,,,,course,,with,,,home,repair,,,,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,,fixing,,some,,,i,would,go,to,the,library,,,and,,start,preparing,for,,before,i,did,,that,
8,,based,,,,,,on,,prices,,,,and,stuff,,,,and,,of,course,were,,for,,,repair,,,,anything,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,,fixing,,some,one,,,would,go,to,,library,,,and,stock,,preparing,for,,before,i,did,,that,
9,,based,,,,,,on,,prices,,,,and,stuff,,,,and,,of,course,were,,for,,,repair,a,thing,,,like,that,i,wouldn't,do,,,,that,many,projects,anyway,so,,i,do,plan,on,building,a,door,,fixing,,some,one,,,would,go,to,the,library,,,and,stock,,preparing,for,,before,i,did,,that,


### Check all sentences remain intact and in original order

In [13]:
for row in range(len(df_aligned)):
    sent_z = df_aligned.iloc[row]
    sent_z = list(filter(None, sent_z))
    sent_z = ' '.join(sent_z)
    print(sent_z == df_temp[row])

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Check shannon entropy of dataframe
i.e. check no columns contain more than one word

In [14]:
from math import log2

def shannon(words):
    words = list(filter(None, words))
    counts = dict()
    for i in words:
        counts[i] = counts.get(i, 0) + 1
    
    total = sum(counts.values())
    entropy_calc = sum(freq / total * log2(total / freq) for freq in counts.values())
    return entropy_calc

# Entropy calc over column

def shannon_col(df, col):
    shannon_calc = shannon(df[col])
    return shannon_calc

# Entropy over data frame

def shannon_total(df):
    total_se = 0
    for col in range(len(df.columns)):
        total_se += shannon(df.iloc[:,col])
    return total_se

In [15]:
shannon_total(df_aligned)

0.0